In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, model_selection, linear_model

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection, CatTargetDriftProfileSection 

# Load data

In [ ]:
iris = datasets.load_iris()
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_frame['target'] = iris.target

# Get Predictions

In [ ]:
train_data, test_data = model_selection.train_test_split(iris_frame, random_state=0)

target = 'target'
prediction = 'prediction'
numerical_features = iris.feature_names
categorical_features = []
features = numerical_features + categorical_features

In [ ]:
model = linear_model.LogisticRegression()
model.fit(train_data[features], train_data.target)

# make a data frame with predictions for each class

train_probas = pd.DataFrame(model.predict_proba(train_data[features]))
train_probas.columns = iris.target_names

test_probas = pd.DataFrame(model.predict_proba(test_data[features]))
test_probas.columns = iris.target_names

In [ ]:
# get labels for target: [0, 1, 0, 2] -> ['setosa', 'versicolor', 'setosa', 'virginica']

train_data['target'] = [iris.target_names[x] for x in train_data['target']]
test_data['target'] = [iris.target_names[x] for x in test_data['target']]

In [ ]:
# merge train and test data with predictions

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

merged_train_data = pd.concat([train_data, train_probas], axis=1)
merged_test_data = pd.concat([test_data, test_probas], axis=1)

In [ ]:
merged_train_data.head()

In [ ]:
iris_column_mapping = ColumnMapping()

iris_column_mapping.target = 'target'
iris_column_mapping.prediction = iris.target_names.tolist()
iris_column_mapping.numerical_features = iris.feature_names

# Target Drift Dashboard full (verbose_level=1)

In [ ]:
iris_prob_classification_dashboard = Dashboard(tabs=[CatTargetDriftTab(verbose_level=1)])
iris_prob_classification_dashboard.calculate(merged_train_data, merged_test_data, 
                                             column_mapping = iris_column_mapping)
iris_prob_classification_dashboard.show()

# Target Drift Dashboard short (verbose_level=0)

In [ ]:
iris_prob_classification_dashboard = Dashboard(tabs=[CatTargetDriftTab(verbose_level=0)])
iris_prob_classification_dashboard.calculate(merged_train_data, merged_test_data, 
                                             column_mapping = iris_column_mapping)
iris_prob_classification_dashboard.show()

# Data Drift Dashboard

In [ ]:
iris_prob_classification_dashboard = Dashboard(tabs=[DataDriftTab()])
iris_prob_classification_dashboard.calculate(merged_train_data, merged_test_data, 
                                             column_mapping = iris_column_mapping)
iris_prob_classification_dashboard.show()

# Target and Data Drift Dashboard

In [ ]:
iris_data_and_target_drift_dashboard = Dashboard(tabs=[DataDriftTab(verbose_level=0), 
                                                       CatTargetDriftTab()])
iris_data_and_target_drift_dashboard.calculate(merged_train_data, merged_test_data, column_mapping=iris_column_mapping)
iris_data_and_target_drift_dashboard.show()

# Profiles

## Target Drift Profile

In [ ]:
iris_target_drift_profile = Profile(sections=[CatTargetDriftProfileSection()])
iris_target_drift_profile.calculate(merged_train_data, merged_test_data, column_mapping=iris_column_mapping) 
iris_target_drift_profile.json()

## Data Drift Profile

In [ ]:
iris_target_drift_profile = Profile(sections=[DataDriftProfileSection()])
iris_target_drift_profile.calculate(merged_train_data, merged_test_data, column_mapping=iris_column_mapping) 
iris_target_drift_profile.json()

## Target and Data Drift Profile

In [ ]:
iris_target_and_data_drift_profile = Profile(sections=[DataDriftProfileSection(), CatTargetDriftProfileSection()])
iris_target_and_data_drift_profile.calculate(merged_train_data, merged_test_data, column_mapping=iris_column_mapping) 
iris_target_and_data_drift_profile.json()